Initialize the environment

In [ ]:
import os
import sys

os.execv(sys.executable, [sys.executable] + sys.argv)

: 

In [1]:
import torch
import argparse

print(torch.__version__)
print(torch.cuda.is_available())

2.8.0.dev20250524+cu128
True


In [2]:
import torch

torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [ ]:
from TamGen_RL import TamGenRL
from utils import prepare_pdb_data, prepare_pdb_data_center, filter_generated_cmpd
import torch

torch.cuda.empty_cache()
torch.cuda.ipc_collect()

# === Setup TamGenRL ===
pdb_id = "3ny8"
print(f"📄 Preparing PDB: {pdb_id}")
prepare_pdb_data(pdb_id)

demo = TamGenRL(
    data="TamGen_Demo_Data",
    ckpt="checkpoints/crossdocked_model/checkpoint_best.pt",
    use_conditional=True
)
demo.reload_data(subset="gen_" + pdb_id.lower())

# === Run Closed-Loop Optimization ===
final_smiles = demo.sample(
    m_sample=50000,         # Number of molecules per iteration
    num_iter=5,           # Number of closed-loop optimization steps
    latent_dim=256,       # Latent space dimensionality (set to your model's config)
    alpha=0.5,            # Centroid shift parameter
    top_k=50,             # How many top molecules to use for shifting
    lambda_sas=0.3,       # Reward hyperparameters
    lambda_logp=0.1,
    lambda_mw=0.1,
    maxseed=20,           # Number of random seeds (first iteration)
    use_cuda=True
)

# === Save or Analyze Results ===
print(f"\nFinal set of SMILES ({len(final_smiles)} molecules):")
for smi in final_smiles:
    print(smi)

📄 Preparing PDB: 3ny8
3ny8 is downloaded
Namespace(no_progress_bar=False, log_interval=1000, log_format=None, tensorboard_logdir='', tbmf_wrapper=False, seed=1, cpu=False, fp16=False, memory_efficient_fp16=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, min_loss_scale=0.0001, threshold_loss_scale=None, user_dir=None, criterion='cross_entropy', tokenizer=None, bpe=None, optimizer='nag', lr_scheduler='fixed', task='translation_coord', num_workers=1, skip_invalid_size_inputs_valid_test=False, max_tokens=1024, max_sentences=None, required_batch_size_multiple=8, dataset_impl=None, gen_subset='gen_8fln', num_shards=1, shard_id=0, path='checkpoints/crossdocked_model/checkpoint_best.pt', remove_bpe=None, quiet=False, model_overrides='{}', results_path=None, beam=20, nbest=20, max_len_a=0, max_len_b=200, min_len=1, match_source_len=False, no_early_stop=False, unnormalized=False, no_beamable_mm=False, lenpen=1, unkpen=0, replace_unk=None, sacrebleu=False, score_refe

  0%|          | 0/20 [00:00<?, ?it/s]/workspace/workspace/TamGen/fairseq/sequence_generator.py:456: UserWarning: An output with one or more elements was resized since it had shape [5, 20], which does not match the required output shape [4, 20]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at /pytorch/aten/src/ATen/native/Resize.cpp:31.)
  torch.topk(
/workspace/workspace/TamGen/fairseq/sequence_generator.py:466: UserWarning: An output with one or more elements was resized since it had shape [5, 20], which does not match the required output shape [4, 20]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered inte

Total valid SMILES and latents after all seeds: 95
Preview of valid SMILES: ['Cc1ccc(OC[C@@H](O)[C@H](C)NC(C)C)c2c1CCC2', 'CC(C)CCC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C[C@@H](O)CC[C@]4(C)[C@H]3CC[C@]12C', 'CCCCCCCC/C=C\\CCCCCCCC(=O)OC[C@H](O)CO', 'CCCCCCCC/C=C\\CCCCCCCC(=O)O', 'OCCOCCOCCO', 'Cc1ccc(OC[C@@H](O)[C@H](C)NC(C)C)c2c1CCC2', 'CC(C)CCC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C[C@@H](O)CC[C@]4(C)[C@H]3CC[C@]12C', 'CCCCCCCC/C=C\\CCCCCCCC(=O)OC[C@H](O)CO', 'CCCCCCCC/C=C\\CCCCCCCC(=O)O', 'OCCOCCOCCO']
Preview of latent vector shapes: [(256,), (256,), (256,), (256,), (256,), (256,), (256,), (256,), (256,), (256,)]
Unique SMILES this iteration: 5 / 95
📊 Optimizing latent space...
[DEBUG] Model created on device: cpu
[DEBUG] z_tensor device: cpu, r_tensor device: cpu
[DEBUG] Cleared z_list and r_list after training.
Model parameters devices: [device(type='cpu'), device(type='cpu'), device(type='cpu'), device(type='cpu')]
Stored tensors devices: []


ValueError: need at least one array to stack

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt

reward_files = sorted(glob.glob('latent_logs/rewards_iter_*.tsv'))
means, maxs, medians = [], [], []

for f in reward_files:
    rewards = []
    with open(f) as fin:
        for line in fin:
            parts = line.strip().split('\t')
            if len(parts) == 2:
                rewards.append(float(parts[1]))
    if rewards:
        rewards = np.array(rewards)
        means.append(rewards.mean())
        maxs.append(rewards.max())
        medians.append(np.median(rewards))

plt.plot(means, label='Mean')
plt.plot(maxs, label='Max')
plt.plot(medians, label='Median')
plt.xlabel('Iteration')
plt.ylabel('Reward')
plt.title('Reward Statistics Across Iterations')
plt.legend()
plt.show()